<span style="font-size: 14pt">MIPT, Advanced ML, Autumn 2017</span>

<span style="font-size: 16pt"> HW #10: Recommender Systems

<span style="color:blue; font-size: 12pt">Andrey Saitgalin </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'"> andrey.saitgalin@gmail.com </span>

<h1 align="center">Organization Info</h1> 

* Дедлайн **18 декабря 23:59** для всех групп.
* В качестве решения задания нужно прислать ноутбук с подробными комментариями (<span style='color:red'> без присланного решения результат контеста не будет засчитан </span>).
* <span style='color:red'>Название команды в контесте должно соответствовать шаблону: НомерГруппы_Имя_Фамилия, например, 494_Ivan_Ivanov</span>.

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall <номер_группы> <фамилия>``, к примеру -- ``ML2017_Aut_fall 401 ivanov``
- Выполненное дз сохраните в файл ``<фамилия>_<группа>_task<номер>.ipnb``, к примеру -- ``ivanov_401_task10.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall Question <Содержание вопроса>``


--------
- **PS1**: Мы используем автоматические фильтры, и просто не найдем ваше дз, если вы не аккуратно его подпишите.
- **PS2**: Напоминаем, что дедлайны жесткие, письма пришедшие после автоматически удаляются =( чтобы соблазна не было.

<h1 align="center">Contest (100%)</h1> 
* Ссылка на контест: https://www.kaggle.com/t/f37413eb448145acba822f3a8c43946d

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix, csr_matrix, csc_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

%matplotlib inline

In [32]:
%%time
data = pd.read_csv("data/train.csv")

CPU times: user 9.2 s, sys: 1.08 s, total: 10.3 s
Wall time: 11.5 s


In [33]:
data.sample(5)

,rating,movieId,userId
3745862,1.0,5842,18394272129399934231
9720865,5.0,258,11906720390964836886
14469885,4.5,1257,2313269538057343768
4155146,5.0,6996,12781698973451016638
14797935,2.5,2615,6550472499606822267


In [4]:
data.groupby("userId").size().min()

9

Переопределим user_id

In [34]:
%%time
old_user_id = [x for x in data["userId"].sort_values().unique()]
new_user_id = {}
for index, value in enumerate(old_user_id, start=1):
    new_user_id[int(value)] = index

CPU times: user 7.04 s, sys: 931 ms, total: 7.97 s
Wall time: 9.12 s


In [35]:
assert len(index_to_user_id_mapping) == 138493

In [36]:
%%time
data = pd.DataFrame.from_records(
    {"movieId": row.movieId, "userId": new_user_id[int(row.userId)], "rating": row.rating}
    for row in data.itertuples())

CPU times: user 1min 17s, sys: 36.4 s, total: 1min 54s
Wall time: 2min 19s


In [9]:
data.head()

,movieId,rating,userId
0,2158,3.0,82143
1,12959,3.5,82143
2,7808,3.5,82143
3,9049,4.0,35749
4,12363,3.0,16069


In [10]:
data['rating'].min(), data['rating'].max()

(0.5, 5.0)

In [11]:
data[["userId", "movieId", "rating"]].to_csv("data/train_2.csv", index=False)

In [12]:
data = pd.read_csv("data/train_2.csv")

Поделим на трейн и тест, выбрав 5 случайных фильмов для каждого юзера

In [13]:
data["random"] = np.random.random(size=(data.shape[0], 1))
idx = data.sort_values(by="random") \
    .reset_index() \
    .groupby("userId") \
    .head(n=5)["index"]

mask = data.index.isin(idx)
train_data = data[~mask]
test_data = data[mask]

Преобразуем рейтинги обратной сигмоидой

In [16]:
EPS = 0.5

def rating_to_score(z):
    return -np.log((4.5 + 2 * EPS) / (z + EPS - 0.5)  - 1)

def score_to_rating(z):
    return 0.5 - EPS + (4.5 + 2 * EPS)  / (1 + np.exp(-z))

In [17]:
train_data['rating'] = rating_to_score(train_data['rating'])
test_data['rating'] = rating_to_score(test_data['rating'])

In [18]:
train_data[["userId", "movieId", "rating"]].to_csv("train_data_2.csv", header=False, index=False)
test_data[["userId", "movieId", "rating"]].to_csv("test_data_2.csv", header=False, index=False)

In [19]:
!awk -F"," '{printf "%f |u %d |i %d\n", $3,$1,$2}' < train_data_2.csv | \
  vw /dev/stdin -b 18 -q ui --rank 30 --l2 0.001 --learning_rate 0.015 \
    --passes 20 --decay_learning_rate 0.97 --power_t 0 \
    --loss_function squared -f movielens.reg --cache_file movielens.cache

creating quadratic features for pairs: ui 
using l2 regularization = 0.001
final_regressor = movielens.reg
Num weight bits = 18
learning rate = 0.015
initial_t = 1
power_t = 0
decay_learning_rate = 0.97
using cache_file = movielens.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0   0.0000   0.0000       63
0.000000 0.000000            2            2.0   0.0000   0.0000       63
0.000000 0.000000            4            4.0   0.0000   0.0000       63
0.000000 0.000000            8            8.0   0.0000   0.0000       63
0.000000 0.000000           16           16.0   0.0000   0.0000       63
0.000000 0.000000           32           32.0   0.0000   0.0000       63
0.000000 0.000000           64           64.0   0.0000   0.0000       63
0.000000 0.000000          128          128.0

In [20]:
!awk -F"," '{printf "%f |u %d |i %d\n", $3,$1,$2}' < data/test_data_2.csv | \
  vw /dev/stdin -i movielens.reg -t -r output_test

creating quadratic features for pairs: ui 
only testing
raw predictions = output_test
Num weight bits = 18
learning rate = 10
initial_t = 1
power_t = 0.5
using no cache
Reading datafile = /dev/stdin
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
2.262248 2.262248            1            1.0   1.5041   0.0000       63
3.782072 5.301897            2            2.0   2.3026   0.0000       63
2.372049 0.962026            4            4.0   0.9808   0.0000       63
3.294489 4.216929            8            8.0   2.3026   0.0000       63
2.035374 0.776259           16           16.0   0.9808   0.0000       63
1.706083 1.376792           32           32.0   0.9808   0.0000       63
1.483096 1.260109           64           64.0   0.1823   0.0000       63
1.352603 1.222110          128          128.0   0.1823   0.0000       63
1.238341 1.124079          256          256.0   0.1823 

In [21]:
output_test_data = pd.read_csv("output_test", header=None)

In [22]:
from sklearn.metrics import mean_squared_error
mean_squared_error(score_to_rating(output_test_data), score_to_rating(test_data['rating'])) ** 0.5

1.2538753452211862

Такой же результат получился на лидерборде

Очень хороший RMSE, должен побить то, что в контесте

In [23]:
result_test_data = pd.read_csv("data/test.csv")
result_test_data = pd.DataFrame.from_records(
    {"movieId": row.movieId, "userId": user_id_to_index_mapping[int(row.userId)]}
    for row in result_test_data.itertuples())

In [24]:
result_test_data.head()

,movieId,userId
0,939,23726
1,19,45416
2,1213,21152
3,1141,106470
4,2935,44653


In [25]:
result_test_data[["userId", "movieId"]].to_csv("result_test_data.csv", header=False, index=False)

In [26]:
!awk -F"," '{printf "0 |u %d |i %d\n", $1,$2}' < data/result_test_data.csv | \
  vw /dev/stdin -i movielens.reg -t -r result.csv

creating quadratic features for pairs: ui 
only testing
raw predictions = result.csv
Num weight bits = 18
learning rate = 10
initial_t = 1
power_t = 0.5
using no cache
Reading datafile = /dev/stdin
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0   0.0000   0.0000       63
0.000000 0.000000            2            2.0   0.0000   0.0000       63
0.000000 0.000000            4            4.0   0.0000   0.0000       63
0.000000 0.000000            8            8.0   0.0000   0.0000       63
0.000000 0.000000           16           16.0   0.0000   0.0000       63
0.000000 0.000000           32           32.0   0.0000   0.0000       63
0.000000 0.000000           64           64.0   0.0000   0.0000       63
0.000000 0.000000          128          128.0   0.0000   0.0000       63
0.000000 0.000000          256          256.0   0.0000  

In [27]:
result = pd.read_csv("result.csv", header=None)

In [69]:
solution[['Id', 'rating']].to_csv("solution.csv", index=None)